In [ ]:
from functools import cache

EXAMPLE = "../example.txt"
INPUT = "../input.txt"

In [ ]:
def get_codes(input_file_name):
    codes = []
    with open(input_file_name, "r") as f:
        for line in f:
            codes.append(line.strip().replace("\n", ""))
    return codes

In [ ]:
codes = get_codes(EXAMPLE)
print(codes)

In [ ]:
# Coordinates of each digit/arrow in their respective keypads (seen as a matrix)
DIGITS = {
    "A": (3, 2),
    "0": (3, 1),
    "1": (2, 0),
    "2": (2, 1),
    "3": (2, 2),
    "4": (1, 0),
    "5": (1, 1),
    "6": (1, 2),
    "7": (0, 0),
    "8": (0, 1),
    "9": (0, 2),
    "FORBIDDEN": (3, 0),
}
ARROWS = {
    "A": (0, 2),
    "^": (0, 1),
    "<": (1, 0),
    "v": (1, 1),
    ">": (1, 2),
    "FORBIDDEN": (0, 0),
}

Let's find the number of vertical and horizontal moves needed to go from one digit/arrow to another


In [ ]:
def get_arrow_moves_between_entities(e1, e2, desc):
    vertical_diff = desc[e2][0] - desc[e1][0]
    horizontal_diff = desc[e2][1] - desc[e1][1]
    nb_of_vertical_moves = abs(vertical_diff)
    nb_of_horizontal_moves = abs(horizontal_diff)
    moves = {"^": 0, "<": 0, "v": 0, ">": 0}
    if vertical_diff <= 0:
        moves["^"] = nb_of_vertical_moves
    else:
        moves["v"] = nb_of_vertical_moves
    if horizontal_diff >= 0:
        moves[">"] = nb_of_horizontal_moves
    else:
        moves["<"] = nb_of_horizontal_moves
    return moves

In [ ]:
@cache
def get_arrow_moves_between_digits(d1, d2):
    return get_arrow_moves_between_entities(d1, d2, DIGITS)


@cache
def get_arrow_moves_between_arrows(a1, a2):
    return get_arrow_moves_between_entities(a1, a2, ARROWS)

In [ ]:
print("0 to 9: ", get_arrow_moves_between_digits("0", "9"))
print("7 to 2: ", get_arrow_moves_between_digits("7", "2"))
print("A to <: ", get_arrow_moves_between_arrows("A", "<"))
print("^ to >: ", get_arrow_moves_between_arrows("^", ">"))

Now we can find all the different ways to make a certain number of horizontal and vertical moves


In [ ]:
def get_possible_sequences(arrow_moves: dict) -> list[str]:
    sequences = []
    if not any([nb > 0 for nb in arrow_moves.values()]):
        return [""]
    for arrow, nb in arrow_moves.items():
        if nb > 0:
            new_arrow_moves = arrow_moves.copy()
            new_arrow_moves[arrow] -= 1
            sequences += [
                arrow + tail for tail in get_possible_sequences(new_arrow_moves)
            ]
    return sequences

In [ ]:
print("0 to 9: ", get_arrow_moves_between_digits("0", "9"))
print(get_possible_sequences(get_arrow_moves_between_digits("0", "9")))
print("7 to 2: ", get_arrow_moves_between_digits("7", "2"))
print(get_possible_sequences(get_arrow_moves_between_digits("7", "2")))
print("3 to 7: ", get_arrow_moves_between_digits("3", "7"))
print(get_possible_sequences(get_arrow_moves_between_digits("3", "7")))

In [ ]:
# The effect of an arrow on coordinates
ARROW_OFFSET = {"^": (-1, 0), "<": (0, -1), "v": (1, 0), ">": (0, 1)}

The digit keypad is the level 0 keypad, the first directional keypad is level 1, etc.

Find all the sequences of buttons to press on the (n+1)-level keypad in order to go from one place to another and press a button on the n-level keypad.


In [ ]:
def get_possible_sequences_between_entities(e1, e2, desc):
    arrow_moves = get_arrow_moves_between_entities(e1, e2, desc)
    sequences = get_possible_sequences(arrow_moves)
    possible_moves = []
    start_row, start_col = desc[e1]
    end_row, end_col = desc[e2]
    for sequence in sequences:
        row, col = start_row, start_col
        for arrow in sequence:
            r, c = row + ARROW_OFFSET[arrow][0], col + ARROW_OFFSET[arrow][1]
            if (r, c) == desc["FORBIDDEN"]:
                break
            row, col = r, c
        if (row, col) == (end_row, end_col):
            # Add A at the end of the sequence because we want the final button to be pressed on the n level keypad
            possible_moves.append(sequence + "A")
    return possible_moves

In [ ]:
@cache
def get_possible_sequences_between_digits(d1, d2):
    return get_possible_sequences_between_entities(d1, d2, DIGITS)


@cache
def get_possible_sequences_between_arrows(a1, a2):
    return get_possible_sequences_between_entities(a1, a2, ARROWS)

In [ ]:
print(get_possible_sequences_between_digits("0", "9"))
print(get_possible_sequences_between_digits("7", "A"))
print(get_possible_sequences_between_arrows("A", "<"))

Find all the ways to press buttons on the level 1 keypad in order to have a certain sequence of button presses on the level 0 keypad


In [ ]:
def get_level1_sequences_for_level0_sequence(l0_seq):
    level_1_sequences = [""]
    i = 0
    while i < len(l0_seq) - 1:
        d1, d2 = l0_seq[i], l0_seq[i + 1]
        l1_seqs = get_possible_sequences_between_entities(d1, d2, DIGITS)
        new_l1_sequences = []
        for current_l1_seq in level_1_sequences:
            for l1_seq in l1_seqs:
                new_l1_sequences.append(current_l1_seq + l1_seq)
        level_1_sequences = new_l1_sequences
        i += 1
    return level_1_sequences

In [ ]:
print(get_level1_sequences_for_level0_sequence("A029A"))

Let's generalize the previous function to go from an n-level sequence to (n+1)-level sequences


In [ ]:
def get_next_level_sequences_for_sequence(seq, desc):
    next_level_sequences = [""]
    i = 0
    while i < len(seq) - 1:
        e1, e2 = seq[i], seq[i + 1]
        next_seqs = get_possible_sequences_between_entities(e1, e2, desc)
        new_sequences = []
        for current_seq in next_level_sequences:
            for next_seq in next_seqs:
                new_sequences.append(current_seq + next_seq)
        next_level_sequences = new_sequences
        i += 1
    return next_level_sequences

In [ ]:
@cache
def get_next_level_sequences_for_sequence_of_digits(seq):
    return get_next_level_sequences_for_sequence(seq, DIGITS)


@cache
def get_next_level_sequences_for_sequence_of_arrows(seq):
    return get_next_level_sequences_for_sequence(seq, ARROWS)

In [ ]:
print(get_next_level_sequences_for_sequence_of_digits("A0"))
print(get_next_level_sequences_for_sequence_of_arrows("A<A"))
print(
    get_next_level_sequences_for_sequence_of_arrows("Av<<A>>^A")
)  # Answer: <vA<AA>>^AvAA<^A>A

Now we can find all the last level sequences that allow the level 0 robot to go from one digit to another and press it


In [ ]:
def get_last_level_sequences_for_first_level_digits(d1, d2):
    seq = d1 + d2
    level1_sequences = get_next_level_sequences_for_sequence_of_digits(seq)
    level2_sequences = []
    for l1_seq in level1_sequences:
        level2_sequences.extend(
            get_next_level_sequences_for_sequence_of_arrows("A" + l1_seq)
        )
    level3_sequences = []
    for l2_seq in level2_sequences:
        level3_sequences.extend(
            get_next_level_sequences_for_sequence_of_arrows("A" + l2_seq)
        )
    return level3_sequences

In [ ]:
print(get_last_level_sequences_for_first_level_digits("A", "0"))
print(get_last_level_sequences_for_first_level_digits("0", "2"))
print(get_last_level_sequences_for_first_level_digits("2", "9"))
print(get_last_level_sequences_for_first_level_digits("9", "A"))

We just need to go through all the digits in the code and find the minimal sequence at the last level to go from one digit to the next


In [ ]:
def get_min_length_for_code(code):
    code = "A" + code
    result = 0
    i = 0
    while i < len(code) - 1:
        d1, d2 = code[i], code[i + 1]
        seqs = get_last_level_sequences_for_first_level_digits(d1, d2)
        min_l = min([len(s) for s in seqs])
        result += min_l
        i += 1
    return result

In [ ]:
for code in codes:
    print(get_min_length_for_code(code))

In [ ]:
def part_1(input_file_name):
    codes = get_codes(input_file_name)
    result = 0
    for code in codes:
        number = int(code[:-1])
        length = get_min_length_for_code(code)
        complexity = number * length
        result += complexity
    print(result)

In [ ]:
part_1(EXAMPLE)

In [ ]:
part_1(INPUT)

Unfortunately, the previous way of doing things doesn't scale to 25 robot controlled directional keypads (or even to 3 for that matter...)

We need a more efficient way of modelizing all this.

The key is to view a sequence of button presses as a set of transitions from one button to another.

For instance, a $(v, A)$ transition means: go from $v$ to $A$ and press $A$.

We'll focus on the directional keypads, the approach to get the possible sequences on the first directional keypad to type in the code doesn't change.

***From now on, level 0 designates the first direcitonal keypad.***


Start by building a set of every possible transitions


In [ ]:
TRANSITIONS = set()
for a1 in ARROWS:
    if a1 == "FORBIDDEN":
        continue
    for a2 in ARROWS:
        if a2 == "FORBIDDEN":
            continue
        TRANSITIONS.add((a1, a2))
print(TRANSITIONS)

Given a particular sequence of buttons, we can generate a dictionary of transitions.

$\{(v, A) : n\}$ means that there are $n$ occurences of the $(v, A)$ transition in the sequence.


In [ ]:
def transitions_from_seq(seq):
    transitions = {}
    i = 0
    while i < len(seq) - 1:
        transition = (seq[i], seq[i + 1])
        if transition in transitions:
            transitions[transition] += 1
        else:
            transitions[transition] = 1
        i += 1
    return transitions

Given a particular transition on an n-level keypad, we can find all the possible dicts of transitions on the (n+1)-level keypad.

On the (n+1)-level keypad, we always start and end at 'A'.


In [ ]:
def get_next_level_transitions_list(transition):
    (a0, a1) = transition
    sequences = get_possible_sequences_between_arrows(a0, a1)
    transitions_list = []
    for seq in sequences:
        transitions = transitions_from_seq("A" + seq)
        transitions_list.append(transitions)
    return transitions_list

The key that will make this approach way faster than the first one is that we can generate in advance how to go from an n-level transition to (n+1)-level transitions.

Since each n-level transition can be achieved using one of several set of (n+1)-level transitions, we generate all the possible complete n-(n+1) mappings (with all combinations of individual n-(n+1) mappings).

Then we can avoid recalculating anything or exploring different paths as we go through the 25 directional keypads.


In [ ]:
possible_mappings = []
for transition in TRANSITIONS:
    next_level_transitions_list = get_next_level_transitions_list(transition)
    updated_possible_mappings = []
    if not next_level_transitions_list:
        continue
    for next_level_transitions in next_level_transitions_list:
        if not possible_mappings:
            # We haven't initialized any mappings
            # We create as many mappings as we have possibilities
            # for the next level transitions for the current transition
            new_mapping = {}
            new_mapping[transition] = next_level_transitions
            updated_possible_mappings.append(new_mapping)
        else:
            # We have already computed mappings for the previous transitions
            # We need to create as many copies of each of them as we have possibilities
            # for the next level transitions for the current transition
            for existing_mapping in possible_mappings:
                # Each existing mapping is copied
                new_mapping = existing_mapping.copy()
                # Updated for the current transition
                new_mapping[transition] = next_level_transitions
                # Added to the list of updated mappings
                updated_possible_mappings.append(new_mapping)
    possible_mappings = updated_possible_mappings

TRANSITION_MAPPINGS = possible_mappings

print(len(TRANSITION_MAPPINGS))
for mapping in TRANSITION_MAPPINGS:
    print(mapping)

Given a dict of transitions for level n, we can now get the dict of transitions for level (n+1), using a particular mapping.


In [ ]:
def get_next_level_transitions(transitions, transition_mapping):
    next_level_transitions = {}
    for transition, i in transitions.items():
        # i is the number of occurences of this transition in our input dict
        for next_transition, j in transition_mapping[transition].items():
            # j is the number of occurences of "next_transition" in the dict for the current transition, in our n-(n+1) mapping
            # We multiply the two numbers to get the number of occurences of "next_transition" in the output dict
            if next_transition in next_level_transitions:
                next_level_transitions[next_transition] += i * j
            else:
                next_level_transitions[next_transition] = i * j
    return next_level_transitions

Let's test it out with the sequences from the example.

Note that we now call level 0 the first directional keypad.

We go through all possible mappings and find the length of each level sequence.

To get the length of the sequence based on the dict of its transitions, we just sum the number of transitions and add 1.


In [ ]:
min_l1 = -1
min_l2 = -1
for mapping in TRANSITION_MAPPINGS:
    level_0 = transitions_from_seq("A<A^A>^^AvvvA")
    level_1 = get_next_level_transitions(level_0, mapping)
    length_1 = sum(level_1.values()) + 1
    if min_l1 == -1 or length_1 < min_l1:
        min_l1 = length_1
    level_2 = get_next_level_transitions(level_1, mapping)
    length_2 = sum(level_2.values()) + 1
    if min_l2 == -1 or length_2 < min_l2:
        min_l2 = length_2

print(min_l1)
print(len("Av<<A>>^A<A>AvA<^AA>A<vAAA>^A"))
print(min_l2)
print(len("A<vA<AA>>^AvAA<^A>A<v<A>>^AvA^A<vA>^A<v<A>^A>AAvA^A<v<A>A>^AAAvA<^A>A"))

Let's generalize the example above and write a function to get the minimum possible length of the final level sequence, based on the level 0 sequence.

As above, we go through all the possible mappings.


In [ ]:
def get_min_length_for_level_0_sequence(level_0_seq, nb_of_dir_keypads):
    min_l = -1
    for mapping in TRANSITION_MAPPINGS:
        # Each robot starts on 'A' so we add it to the sequence, to get the first transition right
        transitions = transitions_from_seq("A" + level_0_seq)
        for _ in range(nb_of_dir_keypads):
            transitions = get_next_level_transitions(transitions, mapping)
        length = sum(transitions.values())
        if min_l == -1 or length < min_l:
            min_l = length
    return min_l

In [ ]:
print(get_min_length_for_level_0_sequence("<A^A>^^AvvvA", 2))

Contrary to part 1, we'll handle the whole code at once and not go from one digit to the next.

So we need to be able to get the level 0 sequence that will allow the robot to type in the whole code.

To do that, we reuse the part 1 function to get the sequence for each transition in digits, and concatenate them.


In [ ]:
def get_level0_sequences_for_code(code):
    code = "A" + code
    i = 0
    sequences = []
    while i < len(code) - 1:
        # Store the level 0 sequences for each transition from one digit to the next
        d1, d2 = code[i], code[i + 1]
        seq = d1 + d2
        level_0_sequences = get_next_level_sequences_for_sequence_of_digits(seq)
        sequences.append(level_0_sequences)
        i += 1
    # Concatenate all level 0 sequences
    for i in range(1, len(sequences)):
        new_sequences = []
        for new_sequence in sequences[i]:
            for old_sequence in sequences[i - 1]:
                new_sequences.append(old_sequence + new_sequence)
        sequences[i] = new_sequences
    return sequences[-1]

In [ ]:
print(get_level0_sequences_for_code("029A"))

Now we can get the minimum length of any sequence that will result in the whole code being entered


In [ ]:
def get_min_length_for_code(code, nb_of_dir_keypads):
    level_0_sequences = get_level0_sequences_for_code(code)
    lengths = []
    for level_0_seq in level_0_sequences:
        min_l = get_min_length_for_level_0_sequence(level_0_seq, nb_of_dir_keypads)
        lengths.append(min_l)
    return min(lengths)

Finally we're able to solve part 2 by reusing the same complexity calculation as for part 1.


In [ ]:
def part_2(input_file_name):
    codes = get_codes(input_file_name)
    result = 0
    for code in codes:
        number = int(code[:-1])
        length = get_min_length_for_code(code, 25)
        complexity = number * length
        result += complexity
    print(result)

In [ ]:
part_2(EXAMPLE)

The result for the example is not provided, which is kind of mean... Let's hope we get the correct result in the input.


In [ ]:
part_2(INPUT)

The result is correct! \o/
